In [1]:
import json
from collections import defaultdict
import string
import re
import numpy as np
import math
import csv

In [2]:
with open('dev.json') as json_file:  
    data = json.load(json_file)

In [7]:
print(data['data'][0])

{'title': '1983-1989: Collaboration with Nikos Karvelas', 'paragraphs': [{'context': 'In May 1983, she married Nikos Karvelas, a composer, with whom she collaborated in 1975 and in November she gave birth to her daughter Sofia. After their marriage, she started a close collaboration with Karvelas. Since 1975, all her releases have become gold or platinum and have included songs by Karvelas. In 1986, she participated at the Cypriot National Final for Eurovision Song Contest with the song Thelo Na Gino Star ("I Want To Be A Star"), taking second place. This song is still unreleased up to date. In 1984, Vissi left her record company EMI Greece and signed with CBS Records Greece, which later became Sony Music Greece, a collaboration that lasted until 2013. In March 1984, she released Na \'Hes Kardia ("If You Had a Heart"). The album was certified gold. The following year her seventh album Kati Simveni ("Something Is Happening") was released which included one of her most famous songs, titl

In [255]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [256]:
num_paragraphs = len(data['data'])
distance_from_prev_question = {}
context_len_id = {}

In [257]:
yes_no = defaultdict(int)
qid_turn_num = defaultdict(int)
qid_ans_len = defaultdict(int)
qid_unans = {}
contextid_qids = defaultdict(list)
qid_to_contextid = defaultdict(int)
qid_to_oringal_ans = {}
total_ques = 0
for i in range(num_paragraphs):
    paragraphs = data['data'][i]['paragraphs']
    context = paragraphs[0]['context']
    context_len_id[i] = len(normalize_answer(context).split())

    qas = paragraphs[0]['qas']
    num_questions = len(qas)
    prev_question_start = 0
    question_ids = []
    prev_ans_text = ''
    for j in range(num_questions):
        total_ques += 1
        qid = qas[j]['id']
        question_ids.append(qid)
        qid_to_contextid[qid] = i
        question_text = qas[j]['question']
        original_ans = qas[j]['orig_answer']
        yesno = qas[j]['yesno']
        
        yes_no[yesno] += 1
        answer_text = original_ans['text']
        qid_to_oringal_ans[qid] = answer_text
        if answer_text == 'CANNOTANSWER':
            qid_unans[qid] = 1
        qid_turn_num[qid] = j 
        qid_ans_len[qid] = len(normalize_answer(answer_text).split())
        
        if j == 0:
            distance_from_prev_question[qid] = -1
            prev_question_start = original_ans['answer_start']
            prev_ans_text = answer_text
        else:
            curr_question_start = original_ans['answer_start']
            distance_from_prev_question[qid] = math.fabs(curr_question_start - prev_question_start)
            prev_question_start = curr_question_start
            prev_ans_text = answer_text
    contextid_qids[i] = question_ids

In [258]:
print(len(distance_from_prev_question))
print(len(qid_unans))
print(len(qid_turn_num))
print(len(qid_ans_len))

7354
1486
7354
7354


In [259]:
import pickle
output = pickle.load(open("pred2.pckl", "rb" ))

In [260]:
predictions = output['predictions']
no_ans_scores = output['no_ans_scores']

In [261]:
qid_to_f1 = {}
unanswerable = {}
qid_to_ans = {}
count = 0
for i in range(len(predictions)):
    single_out = predictions[i]
    no_ans_score = no_ans_scores[i]
    qids = single_out['qid']
    F1s = single_out['F1']
    answers = single_out['best_span_str']
    for j in range(len(qids)):
        qid = qids[j]
        qid_to_f1[qid] = F1s[j]
        if qid_to_oringal_ans[qid] == 'CANNOTANSWER' and answers[j] != 'CANNOTANSWER':
            qid_to_f1[qid] = 0
        if qid_to_oringal_ans[qid] != 'CANNOTANSWER' and answers[j] == 'CANNOTANSWER':
            qid_to_f1[qid] = 0
        if answers[j] == 'CANNOTANSWER':
            unanswerable[qid] = 1
            count += 1
        else:
            unanswerable[qid] = 0
        qid_to_ans[qid] = answers[j]

In [262]:
# answers

In [263]:
num_0_f1 = 0
num_100_f1 = 0
for key in qid_to_f1:
    if qid_to_f1[key] == 0:
        num_0_f1 += 1
    if qid_to_f1[key] == 100:
        num_100_f1 += 1

In [264]:
print(num_0_f1)
print(num_100_f1)

1305
2066


In [265]:
TP = 0
FP = 0
TN = 0
FN = 0
mis_count = 0
for key in unanswerable.keys():
    if unanswerable[key] == 1 and key in qid_unans:
        TP += 1
    if unanswerable[key] == 0 and key in qid_unans:
        if qid_to_f1[key] != 0:
            # print(key)
            # print(qid_to_f1[key])
            qid_to_f1[key] = 0
            mis_count += 1
        FN += 1
    if unanswerable[key] == 1 and key not in qid_unans:
        if qid_to_f1[key] != 0:
            qid_to_f1[key] = 0
            # print(key)
            # print(qid_to_f1[key])
            mis_count += 1
        FP += 1
    if unanswerable[key] == 0 and key not in qid_unans:
        TN += 1

In [266]:
print(mis_count)

0


In [267]:
print(TP)
print(FP)
print(TN)
print(FN)
print(len(qid_unans))

426
100
5768
1060
1486


In [283]:
for qid in qid_to_f1.keys():
    # print(qid)
    f1score = qid_to_f1[qid]
    if qid_to_oringal_ans[qid] != 'CANNOTANSWER':
        answer_len_f1score[qid_ans_len[qid]] += [f1score]
        question_turn_f1score[qid_turn_num[qid]] += [f1score]
        distance_from_prev_ans[distance_from_prev_question[qid]] += [f1score]

    if qid in qid_unans:
        unanswerable_f1score[qid_unans[qid]] += [f1score]

In [281]:
with open('answelen_f1_nounans.csv', mode='w') as csv_file:
    fieldnames = ['answer_length', 'f1']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for key in answer_len_f1score.keys():
        for j in range(len(answer_len_f1score[key])):
            writer.writerow({'answer_length': key, 'f1': answer_len_f1score[key][j]})

In [282]:
with open('question_turn_f1_unans.csv', mode='w') as csv_file:
    fieldnames = ['question_turn', 'f1']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for key in question_turn_f1score.keys():
        for j in range(len(question_turn_f1score[key])):
            writer.writerow({'question_turn': key, 'f1': question_turn_f1score[key][j]})

In [271]:
with open('unans_f1.csv', mode='w') as csv_file:
    fieldnames = ['unanswerable', 'f1']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for key in unanswerable_f1score.keys():
        for j in range(len(unanswerable_f1score[key])):
            writer.writerow({'unanswerable': key, 'f1': unanswerable_f1score[key][j]})

In [279]:
import csv
with open('f1_ques_num.csv', mode='w') as csv_file:
    fieldnames = ['f1', 'question_num']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for key in qid_to_f1.keys():
        if qid_to_oringal_ans[key] != 'CANNOTANSWER':
            writer.writerow({'f1': qid_to_f1[key], 'question_num': 1})

In [273]:
context_len_f1score = defaultdict(list)
for key in context_len_id.keys():
    for j in range(len(contextid_qids[key])):
        context_len_f1score[context_len_id[key]] += [qid_to_f1[contextid_qids[key][j]]]

In [274]:
with open('context_len_f1.csv', mode='w') as csv_file:
    fieldnames = ['context_length', 'f1']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for key in context_len_f1score.keys():
        for j in range(len(context_len_f1score[key])):
            writer.writerow({'context_length': key, 'f1': context_len_f1score[key][j]})

In [284]:
with open('prev_ques_dist_f1_nounans.csv', mode='w') as csv_file:
    fieldnames = ['prev_quesn_distance', 'f1']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for key in distance_from_prev_ans.keys():
        for j in range(len(distance_from_prev_ans[key])):
            if key >= 0:
                writer.writerow({'prev_quesn_distance': key, 'f1': distance_from_prev_ans[key][j]})

In [276]:
import operator
sorted_qid_to_f1 = sorted(qid_to_f1.items(), key=operator.itemgetter(1))

In [277]:
print(len(sorted_qid_to_f1))

7354


In [251]:
count = 0
num_ques_f1 = defaultdict(int)
limit = 10
num_zeros = 0

for key in range(len(sorted_qid_to_f1)):

    if sorted_qid_to_f1[key][1] == 0:
        num_zeros += 1
    if sorted_qid_to_f1[key][1] > limit and sorted_qid_to_f1[key][1]  <= 2*limit:
        num_ques_f1[2*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 2 * limit and sorted_qid_to_f1[key][1]  <= 3*limit:
        num_ques_f1[3*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 3 * limit and sorted_qid_to_f1[key][1]  <= 4*limit:
        num_ques_f1[4*limit] += 1
    
    if sorted_qid_to_f1[key][1] > 4 *limit and sorted_qid_to_f1[key][1]  <= 5*limit:
        num_ques_f1[5*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 5*limit and sorted_qid_to_f1[key][1]  <= 6*limit:
        num_ques_f1[6*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 6*limit and sorted_qid_to_f1[key][1]  <= 7*limit:
        num_ques_f1[7*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 7*limit and sorted_qid_to_f1[key][1]  <= 8*limit:
        num_ques_f1[8*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 8*limit and sorted_qid_to_f1[key][1]  <= 9*limit:
        num_ques_f1[9*limit] += 1
        
    if sorted_qid_to_f1[key][1] > 9*limit and sorted_qid_to_f1[key][1]  <= 10*limit:
        num_ques_f1[10*limit] += 1
print(num_ques_f1) 
print(num_zeros)
val = 0
for key in num_ques_f1.keys():
    val += num_ques_f1[key]
print(val)

defaultdict(<class 'int'>, {20: 630, 30: 297, 40: 131, 50: 120, 60: 199, 70: 279, 80: 458, 90: 693, 100: 2993})
1305
5800


In [ ]:
count = 0

for key in range(len(sorted_qid_to_f1)):
    if sorted_qid_to_f1[key][1] >= 15 and sorted_qid_to_f1[key][1] <=20 :
        num_ques_f1[limit] += 1
        count += 1
        qid = sorted_qid_to_f1[key][0]
        print('{} : {} : {} : {} : {}'.format(qid_to_contextid[qid], qid, qid_to_ans[qid], qid_to_oringal_ans[qid], sorted_qid_to_f1[key][1]))
        if count > 100:
            break

In [195]:
# Find those the old one found as answerable but not the new one
for key in qid_to_f1.keys():
    if key not in qid_unans:
        if unanswerable_old[key] == 1 and unanswerable[key] == 0:
            if(qid_to_f1[key] > 0):
                print(key)

C_0aaa843df0bd467b96e5a496fc0b033d_1_q#1
C_5670d06376064e2c91a316143a3d786a_1_q#1
C_d4307362de854c7e9ccd8f7cffbe6937_0_q#6
C_7c01702c75b94611becc862c15233ff9_1_q#1
C_01e52333288f47fcb1e611c5385ffe29_1_q#5
C_01e52333288f47fcb1e611c5385ffe29_1_q#8
C_01e52333288f47fcb1e611c5385ffe29_0_q#3
C_6abd2040a75d47168a9e4cca9ca3fed5_0_q#6
C_6e1f2e6127634211b25ec6356b3579f5_0_q#0
C_79838e4fce63478fae5bae6e38aff6c1_1_q#2
C_79838e4fce63478fae5bae6e38aff6c1_1_q#4
C_79838e4fce63478fae5bae6e38aff6c1_1_q#8
C_e6af1e489b6d4cc7962abc787e42f116_0_q#2
C_e6af1e489b6d4cc7962abc787e42f116_0_q#3
C_e6642deaec6848118fbc5218004057d5_0_q#4
C_3b1946acb94349afadcf76ed7af2e227_0_q#2
C_cfc2c405e0c24c67b9b023faa38d72ba_0_q#3
C_cfc2c405e0c24c67b9b023faa38d72ba_0_q#10
C_a722bf043d454060b25ef2d81081e05c_1_q#0
C_a722bf043d454060b25ef2d81081e05c_1_q#6
C_4d2e1c130ff8448793116500a399a3f2_0_q#4
C_14dca89984df455a86702f73591901fa_0_q#8
C_f95dda2d52874e7780e36b4d4516e66b_0_q#2
C_daa7a1e6ce8f495494365fc89a387dfb_0_q#1
C_daa7a1e6ce8f4

In [197]:
cid = qid_to_contextid['C_3f72743771a84ca58d380a7f55d87844_1_q#9']
origanswer = qid_to_oringal_ans['C_3f72743771a84ca58d380a7f55d87844_1_q#9']

old_answer = qid_to_ans_old['C_3f72743771a84ca58d380a7f55d87844_1_q#9']
f1_old = qid_to_f1_old['C_3f72743771a84ca58d380a7f55d87844_1_q#9']
new_answer = qid_to_ans['C_3f72743771a84ca58d380a7f55d87844_1_q#9']
f1 = qid_to_f1['C_3f72743771a84ca58d380a7f55d87844_1_q#9']
print(cid)
print('Original Answer: ', origanswer)
print('Old Answer: ', old_answer)
print('New Answet: ', new_answer)
print(f1_old)
print(f1)

115
Original Answer They devised the "Doughnut Dunkers" routine,
They devised the "Doughnut Dunkers
They devised the "Doughnut Dunkers" routine,
0
100.0


In [161]:
data['data'][86]['paragraphs']

[{'context': "On 9 November 2009 he scored 73 runs against New Zealand which was a record for a number 10 batsman in ODI history until Ravi Rampaul surpassed that record with scoring 86* when batting at number 10 position in 2011. In that innings, with his batting partner Saeed Ajmal, Amir added 103 runs to Pakistan's chase, only to fall short by 7 runs. He impressed the Pakistan cricket team, including Wasim Akram, who took credit for discovering Amir, although Akram later commented that Amir needed to gain more weight to aid his development. Later in 2009, Pakistan toured Sri Lanka and in the first One Day International match, he shared a 62 run partnership with fast bowler Umar Gul for the tenth wicket but Pakistan lost by 36 runs. He also played a game against New Zealand in 2009-2010. During a match in the third One Day International match, when Pakistan were reduced to 8 wickets for 86 runs, he scored 73 runs not out, making the highest score by a number 10 batsman in One Day Int